In [ ]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_core.messages import HumanMessage

load_dotenv()

True

In [ ]:
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

agent = create_agent(
    model=llm,
    system_prompt="You are a helpful assistant !",
)

result = agent.invoke(
    {"messages": [HumanMessage(content="how are you?")]}
)

pprint(result)


{'messages': [HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}, id='5231244c-24a7-4b6d-beae-69b7b6688374'),
              AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 54, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CVi7BY4nkMBuu4qrnyebLOzBoUnIv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--1495ae11-9220-4bea-b5d2-6e30d609d220-0', usage_metadata={'input_tokens': 54, 'output_tokens': 29, 'total_tokens': 83, 'input_

In [2]:
try:
    print(result["messages"][1].tool_calls[0]["name"])
except Exception as e:
    pass

print(result["messages"][-1].content)

I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?


In [ ]:
import uuid
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


workflow = StateGraph(state_schema=MessagesState)

load_dotenv()

model = ChatOpenAI(
    model="gpt-4o-mini"
)

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

thread_id = uuid.uuid4()

config = {"configurable": {"thread_id": thread_id}}

input_message = HumanMessage(content="hi, my name is Hosam")

result = app.invoke({"messages": [input_message]}, config)
print(result["messages"][-1].content)

input_message = HumanMessage(content="what was my name?")
result = app.invoke({"messages": [input_message]}, config)
print(result["messages"][-1].content)



{'messages': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='6af92719-01f3-433a-8e08-4e023d6bb15e'),
  AIMessage(content='Hi Bob! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CXAwWX8eLtVk8LYm7PUq8rBMcZO1B', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--dc5685d8-29f6-477d-bd41-58296bc9bf6e-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0

tools

In [6]:
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain.tools import tool
from pydantic import BaseModel, Field

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

def get_weather(city: str, unit: str) -> str:
    """Get weather for a given city with unit."""
    if unit.upper() == "F":
        return f"The weather in {city} is 75°F and sunny."
    return f"The weather in {city} is 24°C and sunny."


agent = create_agent(
    model=llm,
    tools=[get_weather],
    system_prompt="You are a helpful assistant. Use tools only when needed.",
)

result = agent.invoke(
    {"messages": [HumanMessage(content="What's the weather in Dubai in c?")]}
)

pprint(result)
print("\nFinal Answer:", result["messages"][-1].content)


{'messages': [HumanMessage(content="What's the weather in Dubai in c?", additional_kwargs={}, response_metadata={}, id='e214d167-b3f7-4e8d-a915-a9c6ce59c4fb'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 70, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CXAFC5ZOyJZPlbHUtE3ehCcfmwgOc', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--613a894e-ba6a-4a46-aa1c-29d7e2a66174-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Dubai', 'unit': 'c'}, 'id': 'call_glDN9R2yQ7MuCXBmyZPW0EV4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 70, '

In [ ]:
from langchain.tools import StructuredTool
from typing import List, Optional
from pydantic import BaseModel, Field
import requests


token =

headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }


In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.tools import tool
import requests

class SendEmailInput(BaseModel):
    subject: str = Field(..., description="Subject of the email")
    body: str = Field(..., description="Body of the email")
    to: List[str] = Field(..., description="Recipients")
    cc: Optional[List[str]] = Field(None, description="CC recipients")

@tool(args_schema=SendEmailInput)
def send_email(subject: str, body: str, to: List[str], cc: Optional[List[str]] = None) -> str:
    url = "https://graph.microsoft.com/v1.0/me/sendMail"

    message = {
        "subject": subject,
        "body": {"contentType": "Text", "content": body},
        "toRecipients": [{"emailAddress": {"address": addr}} for addr in to],
    }

    if cc:
        message["ccRecipients"] = [{"emailAddress": {"address": addr}} for addr in cc]

    email_data = {"message": message, "saveToSentItems": True}

    res = requests.post(url, headers=headers, json=email_data)

    if res.status_code == 202:
        return f"Email sent to {', '.join(to)}"
    return f"Failed: {res.text}"


In [ ]:
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_agent(
    model=llm,
    tools=[send_email],
    system_prompt="You are a helpful assistant."
)

result = agent.invoke(
    {"messages": [HumanMessage(content="Send email to saying Hello hosam_z9@hotmail.com")]}
)

pprint(result)
print("Final:", result["messages"][-1].content)
